In [ ]:
import fingerprint
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.util import invert
from gtda.homology import VietorisRipsPersistence

In [ ]:
# Binarizing finger print image
original_img = cv2.imread("images/print1.png", cv2.IMREAD_ANYCOLOR)

BnW_image = fingerprint.BinarizeFingerprint(original_img, 
                                            method='Pixel Blocks Optimized', 
                                            blockSize=15)

BnW_image = invert(BnW_image)

# Plotting
plt.imshow(BnW_image, cmap='gray')
plt.show()

In [ ]:
points = fingerprint.ToPointCloud(BnW_image)

#plotting
from gtda.plotting import plot_point_cloud
plot_point_cloud(points[0])

In [ ]:
persistence = VietorisRipsPersistence(
    metric="euclidean",
    homology_dimensions=[0, 1], # Track connected components, loops
    n_jobs=6
)

diagrams_basic = persistence.fit_transform(points)

In [ ]:
from gtda.plotting import plot_diagram

plot_diagram(diagrams_basic[0])

# Another Attempt based on MNIST

In [1]:
import fingerprint
import cv2
from skimage.util import invert
from gtda.images import RadialFiltration
from gtda.diagrams import Scaler
from gtda.diagrams import HeatKernel

original_img = cv2.imread("images/print1.png", cv2.IMREAD_ANYCOLOR)
BnW_image = fingerprint.BinarizeFingerprint(original_img)
radial_filtration = RadialFiltration(center=np.array([150, 50]))
filtered = radial_filtration.fit_transform(np.array([invert(BnW_image)]))
radial_filtration.plot(filtered, colorscale="jet")


In [ ]:
diagrams_basic = fingerprint.ToPersistenceDiagram(filtered)
scaler = Scaler()
scaled_diagrams = scaler.fit_transform(diagrams_basic)
scaler.plot(scaled_diagrams)

In [ ]:
h = HeatKernel(sigma=.15, n_bins=60, n_jobs=-1)
h_img = h.fit_transform(scaled_diagrams)
h.plot(h_img, homology_dimension_idx=1, colorscale='jet')